In [1]:
import datetime
import random
import tensorflow as tf
import pandas as pd
import numpy as np

from configparser import ConfigParser
from tqdm.auto import tqdm
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorboard.plugins.hparams import api as hp
from utils import get_config, create_data_splits, OrderedDictWithDefaultList

# Read config.ini file
SETTINGS, COLOURS, EYETRACKER, TF = get_config("config.ini")
np.set_printoptions(precision=4)

# Read and process the data

In [2]:
def load_data():
    df = pd.read_csv("data/positions.csv")
    df["targets"] = list(zip(df["x"], df["y"]))
    df["filename"] = df["id"].astype("str") + ".jpg"
    filenames = df["filename"].tolist()

    data = OrderedDictWithDefaultList()
    data["targets"] = np.array(df["targets"].tolist(), dtype="float32")
    data["head_angle"] = np.array(df["head_angle"], dtype="float32")
    img_types = ["face", "face_aligned", "l_eye", "r_eye", "head_pos"]

    for f in tqdm(filenames):
        for img_type in img_types:
            cur_img = load_img("data/{}/{}".format(img_type, f))
            data[img_type].append(img_to_array(cur_img))

    for img_type in img_types:
        data[img_type] = np.array(data[img_type], dtype="float32") / 255.0

    return create_data_splits(data)

data = load_data()

100%|██████████| 15544/15544 [00:31<00:00, 486.45it/s]


In [ ]:
# # TF data pipeline test
# def create_datasets(*desired_data):
#     all_features = []
#     data_list = list(desired_data)

#     # Get xy targets
#     df = pd.read_csv("data/positions.csv", sep=",")
#     xy_targets = list(zip(df["x"], df["y"]))
#     d_targets = tf.data.Dataset.from_tensor_slices(xy_targets)
#     print("targets : ", d_targets)

#     # Get head_angle value if specified
#     if "head_angle" in data_list:
#         data_list.remove("head_angle")
#         d_head_angle = tf.data.Dataset.from_tensor_slices(
#             df["head_angle"].astype(np.float32)
#         )
#         all_features.append(d_head_angle)
#         print("head_angle : ", d_head_angle)

#     # Get requested images
#     for d in data_list:
#         img_list = tf.data.Dataset.list_files("data/{}/*".format(d), shuffle=False)
#         img_list = img_list.map(
#             process_image, num_parallel_calls=tf.data.experimental.AUTOTUNE
#         )
#         all_features.append(img_list)
#         print(d, ": ", img_list)

#     # Combine inputs with targets ((a,b,c), (x,y))
#     all_features = tf.data.Dataset.zip(tuple(all_features))
#     ds = tf.data.Dataset.zip((all_features, d_targets))
#     ds = ds.shuffle(1000, reshuffle_each_iteration=False)

#     # Split into train/val/test (80/10/10)
#     d_train, d_val, d_test = split_ds(ds)

#     return {
#         "train": optimize_ds(d_train),
#         "val": optimize_ds(d_val),
#         "test": optimize_ds(d_test),
#     }


# def process_image(path):
#     img = tf.io.read_file(path)
#     img = tf.image.decode_jpeg(img)
#     img = tf.image.convert_image_dtype(img, tf.float32)

#     return img


# def split_ds(ds, train=0.8, val=0.1, test=0.1):
#     assert train + val + test == 1.0

#     ds_size = tf.data.experimental.cardinality(ds).numpy()
#     train_size = int(train * ds_size)
#     val_size = int(val * ds_size)
#     test_size = int(test * ds_size)

#     d_train = ds.take(train_size)
#     d_test = ds.skip(train_size)
#     d_val = d_test.take(val_size)
#     d_test = d_test.skip(val_size)

#     return d_train, d_val, d_test


# def optimize_ds(ds, batch_size=128):
#     ds = ds.cache()
#     ds = ds.shuffle(buffer_size=1000)
#     ds = ds.batch(batch_size)
#     ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

#     return ds


# Model experiments
## Basic sequential model using a single face image

In [3]:
HP_OPTIMIZER = hp.HParam("optimizer", hp.Discrete(["adam", "rmsprop", "adagrad"]))
HP_CONV_KERNEL_SIZE = hp.HParam("conv_kernel_size", hp.Discrete([3, 5, 7, 9]))
HP_CONV_LAYERS = hp.HParam("conv_layers", hp.IntInterval(1, 5))
HP_DENSE_LAYERS = hp.HParam("dense_layers", hp.IntInterval(1, 3))
HP_DROPOUT = hp.HParam("dropout", hp.RealInterval(0.1, 0.4))

HPARAMS = [
    HP_OPTIMIZER,
    HP_CONV_KERNEL_SIZE,
    HP_CONV_LAYERS,
    HP_DENSE_LAYERS,
    HP_DROPOUT,
]

METRICS = [
    hp.Metric(
        "epoch_loss",
        group="validation",
        display_name="loss (val)",
    ),
    hp.Metric(
        "batch_loss",
        group="train",
        display_name="loss (train)",
    ),
]

def build_model(hparams, seed=87):
    rng = random.Random(seed)

    model = models.Sequential()

    # Add convolutional layers
    conv_filters = 8
    model.add(layers.Conv2D(conv_filters, hparams[HP_CONV_KERNEL_SIZE], activation='relu', input_shape=(SETTINGS['image_size'], SETTINGS['image_size'], 3)))
    model.add(layers.MaxPool2D(pool_size=2, padding="same"))

    for _ in range(hparams[HP_CONV_LAYERS]):
        model.add(
            layers.Conv2D(
                filters=conv_filters,
                kernel_size=hparams[HP_CONV_KERNEL_SIZE],
                padding="same",
                activation="relu",
            )
        )
        model.add(layers.MaxPool2D(pool_size=2, padding="same"))
        conv_filters *= 2

    model.add(layers.Flatten())
    model.add(layers.Dropout(hparams[HP_DROPOUT], seed=rng.random()))

    # Add fully connected layers
    dense_neurons = 32
    for _ in range(hparams[HP_DENSE_LAYERS]):
        model.add(layers.Dense(dense_neurons, activation="relu"))
        dense_neurons *= 2

    # Add the final output layer
    model.add(layers.Dense(2))

    model.compile(
        loss=tf.keras.losses.MeanSquaredError(),
        optimizer=hparams[HP_OPTIMIZER],
        metrics=[tf.keras.metrics.MeanSquaredError()],
    )
    return model

def run_single(data, base_logdir, session_id, hparams, best_loss):
    tf.keras.backend.clear_session()

    model = build_model(hparams=hparams, seed=session_id)

    logdir = os.path.join(base_logdir, session_id)
    tb_callback = tf.keras.callbacks.TensorBoard(
        logdir,
        histogram_freq=0,
        update_freq=500,
        write_graph=True,
        write_images=True,
        profile_batch=0,
    )
    hparams_callback = hp.KerasCallback(logdir, hparams)

    history = model.fit(
        x=data['face']['train'],
        y=data['targets']['train'],
        epochs=2,
        shuffle=False,
        validation_data=(data['face']['val'], data['targets']['val']),
        callbacks=[tb_callback, hparams_callback],
        verbose=1
    )

    results = model.evaluate(x=data['face']['test'], y=data['targets']['test'], batch_size=None, verbose=0)
    if results[1] < best_loss:
        print(results)
        print("Saving new model. Loss: {}".format(round(results[1]*100, 3)))
        model.save('models/eyetracking_model.h5')
        return results[1]
    else:
        return best_loss


def run_all(logdir, data, verbose=True):
    best_loss = TF['best_loss']
    rng = random.Random(0)

    with tf.summary.create_file_writer(logdir).as_default():
        hp.hparams_config(hparams=HPARAMS, metrics=METRICS)

    num_experiments = 2
    session_index = 0
    for group_index in range(num_experiments):
        hparams = {h: h.domain.sample_uniform(rng) for h in HPARAMS}
        hparams_string = str(hparams)

        session_id = str(session_index)
        session_index += 1
        print("--- Running experiment {}/{}".format(session_index, num_experiments))

        if verbose:
            print(hparams_string)
        
        best_loss = run_single(
            data=data,
            base_logdir=logdir,
            session_id=session_id,
            hparams=hparams,
            best_loss=best_loss,
        )

    # Update the best accuracy - temporary (dont choose models based on test performance)
    config = ConfigParser()
    config.read("config.ini")
    config_file = open("config.ini", "w")
    config.set("TF", "best_loss", str(best_loss))
    config.write(config_file)
    config_file.close() 

np.random.seed(0)
model_name = datetime.datetime.now().strftime("face %Y_%m_%d %H_%M_%S")
logdir = 'logs/{}'.format(model_name)
run_all(logdir=logdir, data=data, verbose=True)


--- Running experiment 1/2
{HParam(name='optimizer', domain=Discrete(['adagrad', 'adam', 'rmsprop']), display_name=None, description=None): 'adam', HParam(name='conv_kernel_size', domain=Discrete([3, 5, 7, 9]), display_name=None, description=None): 9, HParam(name='conv_layers', domain=IntInterval(1, 5), display_name=None, description=None): 1, HParam(name='dense_layers', domain=IntInterval(1, 3), display_name=None, description=None): 2, HParam(name='dropout', domain=RealInterval(0.1, 0.4), display_name=None, description=None): 0.38963946590857523}
Epoch 1/2


UnknownError: 2 root error(s) found.
  (0) Unknown:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv2d/Relu (defined at <ipython-input-3-5847724ccfce>:85) ]]
	 [[ReadVariableOp/_26]]
  (1) Unknown:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv2d/Relu (defined at <ipython-input-3-5847724ccfce>:85) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_1086]

Function call stack:
train_function -> train_function


## Multiple input, multiple data types

In [11]:
HP_OPTIMIZER = hp.HParam("optimizer", hp.Discrete(["adam", "rmsprop", "adagrad"]))
HP_CONV_KERNEL_SIZE = hp.HParam("conv_kernel_size", hp.Discrete([3, 5, 7, 9]))
HP_CONV_LAYERS = hp.HParam("conv_layers", hp.IntInterval(1, 5))
HP_DENSE_LAYERS = hp.HParam("dense_layers", hp.IntInterval(1, 3))
HP_DROPOUT = hp.HParam("dropout", hp.RealInterval(0.1, 0.4))

HPARAMS = [
    HP_OPTIMIZER,
    HP_CONV_KERNEL_SIZE,
    HP_CONV_LAYERS,
    HP_DENSE_LAYERS,
    HP_DROPOUT,
]

METRICS = [
    hp.Metric(
        "epoch_loss",
        group="validation",
        display_name="loss (val)",
    ),
    hp.Metric(
        "batch_loss",
        group="train",
        display_name="loss (train)",
    ),
]

def build_model(hparams, seed=87):
    face_input = layers.Input(shape=(SETTINGS['image_size'], SETTINGS['image_size'], 3), name="face")
    l_eye_input = layers.Input(shape=(SETTINGS['image_size'], SETTINGS['image_size'], 3), name="l_eye")
    r_eye_input = layers.Input(shape=(SETTINGS['image_size'], SETTINGS['image_size'], 3), name="r_eye")

    conv_filters = 8
    face = layers.Conv2D(filters=conv_filters, kernel_size=3, activation='relu')(face_input)
    face = layers.MaxPool2D(pool_size=2, padding="same")(face)

    for _ in range(hparams[HP_CONV_LAYERS]):
        face = layers.Conv2D(filters=conv_filters, kernel_size=3, activation='relu')(face)
        face = layers.MaxPool2D(pool_size=2, padding="same")(face)
        conv_filters *= 2

    face = layers.Flatten()(face)
    face = layers.Dense(16, activation="relu")(face)
    #face_model = tf.keras.Model(inputs=face_input, outputs=face, name="face_model")

    l_eye = layers.Conv2D(filters=32, kernel_size=3, activation='relu')(l_eye_input)
    l_eye = layers.MaxPool2D(pool_size=2, padding="same")(l_eye)
    l_eye = layers.Flatten()(l_eye)
    l_eye = layers.Dense(16, activation="relu")(l_eye)
    #l_eye_model = tf.keras.Model(inputs=l_eye_input, outputs=l_eye, name="l_eye_model")

    r_eye = layers.Conv2D(filters=32, kernel_size=3, activation='relu')(r_eye_input)
    r_eye = layers.MaxPool2D(pool_size=2, padding="same")(r_eye)
    r_eye = layers.Flatten()(r_eye)
    r_eye = layers.Dense(16, activation="relu")(r_eye)
    #r_eye_model = tf.keras.Model(inputs=r_eye_input, outputs=r_eye, name="r_eye_model")

    combined = layers.concatenate([face, l_eye, r_eye])

    dense_neurons = 16
    output = layers.Dense(dense_neurons, activation="relu")(combined)

    for _ in range(hparams[HP_DENSE_LAYERS]):
        output = layers.Dense(dense_neurons, activation="relu")(output)
        dense_neurons *= 2

    output = layers.Dense(2)(output)

    model = tf.keras.Model(inputs=[face_input, l_eye_input, r_eye_input], outputs=output)
    model.compile(
        loss=tf.keras.losses.MeanSquaredError(),
        optimizer="adam",
        metrics=[tf.keras.metrics.MeanSquaredError()],
    )

    return model

def run_single(data, base_logdir, session_id, hparams):
    tf.keras.backend.clear_session()

    model = build_model(hparams=hparams, seed=session_id)

    logdir = os.path.join(base_logdir, session_id)
    tb_callback = tf.keras.callbacks.TensorBoard(
        logdir,
        histogram_freq=0,
        update_freq=500,
        write_graph=True,
        write_images=True,
        profile_batch=0,
    )
    hparams_callback = hp.KerasCallback(logdir, hparams)

    history = model.fit(
            x=[data['face']['train'], data['l_eye']['train'], data['r_eye']['train']],
            y=data['targets']['train'],
            batch_size=128,
            epochs=2,
            validation_data=([data['face']['val'], data['l_eye']['val'], data['r_eye']['val']], data['targets']['val']),
            callbacks=[tb_callback, hparams_callback],
            verbose=1
    )

def run_all(logdir, data, verbose=True):
    rng = random.Random(0)

    with tf.summary.create_file_writer(logdir).as_default():
        hp.hparams_config(hparams=HPARAMS, metrics=METRICS)

    num_experiments = 2
    session_index = 0
    for group_index in range(num_experiments):
        hparams = {h: h.domain.sample_uniform(rng) for h in HPARAMS}
        hparams_string = str(hparams)

        session_id = str(session_index)
        session_index += 1
        print("--- Running experiment {}/{}".format(session_index, num_experiments))

        if verbose:
            print(hparams_string)
        
        run_single(
            data=data,
            base_logdir=logdir,
            session_id=session_id,
            hparams=hparams
        )

np.random.seed(0)
model_name = datetime.datetime.now().strftime("full %Y_%m_%d %H_%M_%S")
logdir = 'logs/{}'.format(model_name)
run_all(logdir=logdir, data=data, verbose=True)


--- Running experiment 1/2
{HParam(name='optimizer', domain=Discrete(['adagrad', 'adam', 'rmsprop']), display_name=None, description=None): 'adam', HParam(name='conv_kernel_size', domain=Discrete([3, 5, 7, 9]), display_name=None, description=None): 9, HParam(name='conv_layers', domain=IntInterval(1, 5), display_name=None, description=None): 1, HParam(name='dense_layers', domain=IntInterval(1, 3), display_name=None, description=None): 2, HParam(name='dropout', domain=RealInterval(0.1, 0.4), display_name=None, description=None): 0.38963946590857523}
Epoch 1/2
98/98 [==============================] - 5s 51ms/step - loss: 495909.7500 - accuracy: 0.7529 - val_loss: 155016.6562 - val_accuracy: 0.7529
Epoch 2/2
98/98 [==============================] - 5s 47ms/step - loss: 150835.9219 - accuracy: 0.7529 - val_loss: 145008.0781 - val_accuracy: 0.7529
--- Running experiment 2/2
{HParam(name='optimizer', domain=Discrete(['adagrad', 'adam', 'rmsprop']), display_name=None, description=None): 'adam'